In [1]:
import os
import kagglehub
import json
import pandas as pd
from trl import SFTTrainer, SFTConfig
from evaluate import load as load_metric

os.environ["USE_FLASH_ATTENTION_2"] = "false"

from unsloth import FastLanguageModel
from transformers import TrainingArguments
from datasets import load_dataset, Dataset
import torch

C:\Users\erenc\AppData\Local\Temp\ipykernel_4008\1673039938.py:10: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.7.0+cu118)
    Python  3.10.11 (you have 3.10.6)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
"cuda" if torch.cuda.is_available() else "cpu"

'cuda'

In [3]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Model Training (Finetuning)

In [ ]:
model_name = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit" 

# Load model with LoRA enabled
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "o_proj", "v_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

## Dataset Preparation

In [5]:
dataset = load_dataset("wikisql", trust_remote_code=True)

In [6]:
dataset

DatasetDict({
    test: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 15878
    })
    validation: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 8421
    })
    train: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 56355
    })
})

In [7]:
dataset["train"]["question"][0]

'Tell me what the notes are for South Australia '

In [8]:
dataset["train"]["sql"][0]

{'human_readable': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA',
 'sel': 5,
 'agg': 0,
 'conds': {'column_index': [3],
  'operator_index': [0],
  'condition': ['SOUTH AUSTRALIA']}}

In [9]:
train_dataset, eval_dataset, test_dataset = dataset["train"], dataset["validation"], dataset["test"]

In [10]:
# Preprocessing function
def preprocess_example(example):
    headers = example["table"]["header"]
    types = example["table"]["types"]
    table_str = " | ".join([f"{h} ({t})" for h, t in zip(headers, types)])
    return {
        "question": example["question"],
        "sql": example["sql"]["human_readable"],
        "table_str": table_str
    }

# Apply preprocessing
train_processed = train_dataset.map(preprocess_example)
eval_processed = eval_dataset.map(preprocess_example)
test_processed = test_dataset.map(preprocess_example)

In [11]:
bleu_metric = load_metric("bleu")

# I added the compute metrics function here but I don't use it in the below code due to the Out of Memory error I get
# but it should work 
def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    if isinstance(predictions, tuple):
        predictions = predictions[0]

    predictions = predictions.tolist() if hasattr(predictions, 'tolist') else predictions
    labels = labels.tolist() if hasattr(labels, 'tolist') else labels

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    def clean(text):
        return text.replace("query=", "").strip(", \n").lower()

    decoded_preds = [clean(p) for p in decoded_preds]
    decoded_labels = [clean(l) for l in decoded_labels]

    decoded_preds = [p.split() for p in decoded_preds]
    decoded_labels = [[l.split()] for l in decoded_labels]

    bleu = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"eval_bleu": bleu["bleu"]}



def formatting_func(batch):
    prompts = []
    for question, sql_str, table_str in zip(batch["question"], batch["sql"], batch["table_str"]):
        prompt = tokenizer.apply_chat_template(
            [
                {"role": "user", "content":
                    f"""You are a SQL expert.
                        
                        Given the question, original query, generate a SQL query to answer the question. Follow the response format and guidelines strictly. Do not include any additional text outside the specified format.

                        Use the table schema below!
                        ===Tables===
                        {table_str}
                        

                        ===Response Guidelines===
                        1. Ensure the SQL is properly formatted.
                        2. Always return a valid JSON object using the structure below.
                        
                        ===Response Format===
                        query=<SQL query if sufficient context is available>,
                        
                        ===Question===
                        {question}
                    """
                },
                {"role": "assistant", "content": f"query={sql_str}"}
            ],
            tokenize=False,
            add_generation_prompt=False
        )
        prompts.append(prompt)

    return prompts




trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_processed,
    eval_dataset=eval_processed.select(range(20)),
    formatting_func=formatting_func,
    # compute_metrics=compute_metrics,
    args=SFTConfig(
        per_device_train_batch_size=2,
        dataset_num_proc=1, 
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=30,
        learning_rate=2e-4,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
        push_to_hub=True,
        hub_model_id="nerzid/qwen2.5-3B-4bit-text2sql",
        hub_private_repo=False,
        eval_strategy="steps",
        eval_steps=1,
    )
)

C:\Users\erenc\AppData\Local\Temp\ipykernel_4008\3436951744.py:67: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Applying formatting function to train dataset:   0%|          | 0/56355 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/56355 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/56355 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/56355 [00:00<?, ? examples/s]

...

Applying formatting function to eval dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/20 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

In [12]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 56,355 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 7,372,800/3,000,000,000 (0.25% trained)


Step,Training Loss,Validation Loss
1,2.790200,4.943738
2,2.987100,4.927110
3,2.585400,4.888593
4,2.053900,4.820020
5,1.415700,4.711935
6,0.638300,4.572373
7,0.373800,4.442413
8,0.441200,4.338674
9,0.396500,4.246625
10,0.292000,4.173190


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=30, training_loss=0.5132651489848892, metrics={'train_runtime': 826.5704, 'train_samples_per_second': 0.29, 'train_steps_per_second': 0.036, 'total_flos': 916973249003520.0, 'train_loss': 0.5132651489848892})

In [5]:
trainer.evaluate()

Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


{'eval_loss': 4.7642130851745605,
 'eval_runtime': 1961.1596,
 'eval_samples_per_second': 4.294,
 'eval_steps_per_second': 1.074}

In [13]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/nerzid/qwen2.5-3B-4bit-text2sql/commit/cc03f4d218ef914c972f0985c32aaa1810c59db6', commit_message='End of training', commit_description='', oid='cc03f4d218ef914c972f0985c32aaa1810c59db6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nerzid/qwen2.5-3B-4bit-text2sql', endpoint='https://huggingface.co', repo_type='model', repo_id='nerzid/qwen2.5-3B-4bit-text2sql'), pr_revision=None, pr_num=None)

# Finetuned Model Evaluation

In [55]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from evaluate import load as load_metric
from tqdm import tqdm

# Load the fine tuned model and tokenizer
model_id = "nerzid/qwen2.5-3B-4bit-text2sql"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
model.eval().cuda()  # or use .to("cuda") / .to("cpu") depending on device

# Load evaluation data (100 samples for test)
dataset = load_dataset("wikisql", split="validation[:10]")

# BLEU evaluator
bleu = load_metric("bleu")

# Clean the text if it doesn't output the sql query correctly
def clean(text):
    if "query:" in text:
        return text.split("query:")[-1].lstrip()
    elif "query=" in text:
        return text.split("query=")[-1].lstrip()
    else:
        return text

# Evaluation
predictions = []
references = []

for example in tqdm(dataset):
    question = example["question"]
    ground_truth_sql = example["sql"]["human_readable"]
    headers = example["table"]["header"]
    types = example["table"]["types"]
    table_str = " | ".join([f"{h} ({t})" for h, t in zip(headers, types)])
    # Create prompt
    prompt = tokenizer.apply_chat_template(
        [
            {"role": "user", "content":
                    f"""You are a SQL expert.
                        
                        Given the question, original query, generate a SQL query to answer the question. Follow the response format and guidelines strictly. Do not include any additional text outside the specified format.

                        Use the table schema below!
                        ===Tables===
                        {table_str}
                        
                        ===Response Guidelines===
                        1. Ensure the SQL is properly formatted.
                        2. Always return a valid JSON object using the structure below.
                        
                        ===Response Format===
                        query=<SQL Query>,
                        
                        ===Question===
                        {question}
                    """
            }
            ,
        ],
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode and clean
    pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pred_clean = clean(pred_text)
    gold_clean = ground_truth_sql

    # Add to metric buffers
    predictions.append(pred_clean)
    references.append([gold_clean])  # BLEU expects list of list

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [08:10<00:00, 49.00s/it]


In [56]:
# 📊 Compute BLEU
results = bleu.compute(predictions=predictions, references=references)
print(f"\n🔍 BLEU score: {results['bleu'] * 100:.2f}")


🔍 BLEU score: 18.18


In [57]:
i = 1
question = dataset[i]["question"]
print(f"Question: {question}\nSQL: {references[i]}\nPredSQL: {predictions[i]}")

Question: How many schools did player number 3 play at?
SQL: ['SELECT COUNT School/Club Team FROM table WHERE No. = 3']
PredSQL: {"sql":"SELECT COUNT(DISTINCT School/Club Team) FROM Player WHERE No. = '3'"}


In [ ]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Baseline Model Evaluation

In [58]:
# Loading the model
model_id = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
model.eval().cuda()

# Evaluation
predictions = []
references = []

for example in tqdm(dataset):
    question = example["question"]
    ground_truth_sql = example["sql"]["human_readable"]
    headers = example["table"]["header"]
    types = example["table"]["types"]
    table_str = " | ".join([f"{h} ({t})" for h, t in zip(headers, types)])
    # Create prompt
    prompt = tokenizer.apply_chat_template(
        [
            {"role": "user", "content":
                    f"""You are a SQL expert.
                        
                        Given the question, original query, generate a SQL query to answer the question. Follow the response format and guidelines strictly. Do not include any additional text outside the specified format.

                        Use the table schema below!
                        ===Tables===
                        {table_str}
                        
                        ===Response Guidelines===
                        1. Ensure the SQL is properly formatted.
                        2. Always return a valid JSON object using the structure below.
                        
                        ===Response Format===
                        query=<SQL Query>,
                        
                        ===Question===
                        {question}
                    """
            }
            ,
        ],
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode and clean
    pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pred_clean = clean(pred_text)
    gold_clean = ground_truth_sql

    # Add to metric buffers
    predictions.append(pred_clean)
    references.append([gold_clean])  # BLEU expects list of list

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:47<00:00, 10.77s/it]


In [59]:
# 📊 Compute BLEU
results = bleu.compute(predictions=predictions, references=references)
print(f"\n🔍 BLEU score: {results['bleu'] * 100:.2f}")


🔍 BLEU score: 17.94


In [64]:
i = 1
question = dataset[i]["question"]
print(f"Question: {question}\nSQL: {references[i]}\nPredSQL: {predictions[i]}")

Question: How many schools did player number 3 play at?
SQL: ['SELECT COUNT School/Club Team FROM table WHERE No. = 3']
PredSQL: SELECT COUNT(DISTINCT School_Club_Team) FROM Player WHERE No__ = '3'
